# Analyzing CIA Factbook Data Using SQL
## Introduction

In this project, SQL will be introduced in jupyter notebooks to analyze a slice of data from the CIA Factbook.

A variety of queries, subqueries and data aggregation techniques will be used to explore information about this data. 

This project was intially hosted on an online virtual environment which is reflected in the first cell. 

Below is a brief description of the tailored data.

## Data Documentation
`name` — the name of the country

`area` — the country's total area (both land and water)

`area_land`  — the country's land area in square kilometers

`area_water` — the country's waterarea in square kilometers

`population` — the country's population

`population_growth` — the country's population growth as a percentage

`birth_rate` — the country's birth rate, or the number of births per year per 1,000 people

`death_rate` — the country's death rate, or the number of death per year per 1,000 people
### Connecting to Factbook Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

### Overview of the data
Return information on the tables in the database.

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Return information from the `facts` table in the database.

In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Summary Statistics
We'll begin by calculating some summary statistics and look for any outlier countries.

In [4]:
%%sql
SELECT MIN(population) min_population, 
       MAX(population) max_population, 
       MIN(population_growth) min_pop_growth, 
       MAX(population_growth) max_pop_growth
FROM facts;

Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


### Exploring Outliers
Populations of `0` and `7,256,490,011` seem like outliers and we should dynamically find which countries they belong to.

In [5]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population) FROM facts)
OR population = (SELECT MAX(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We see that there is a row for the `World` that assumingly sums all the population.
There is also `Antarctica` where there are no natural inhabitants.

Recalculate the summary statstics discarding the `World` row.

In [6]:
%%sql
SELECT MIN(population) min_population, 
       MAX(population) max_population,
       MIN(population_growth) min_pop_growth,
       MAX(population_growth) max_pop_growth
FROM facts
WHERE name <> 'World';

Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


### Exploring Average Population and Area
Let's explore population density remembering to exclude the `World` row.

In [7]:
%%sql
SELECT ROUND(AVG(population), 2) avg_population, 
       ROUND(AVG(area), 2) avg_area_sq_km
FROM facts
WHERE name <> 'World';

Done.


avg_population,avg_area_sq_km
32242666.57,555093.55


### Finding Densely Populated Countries 
Dynamically identify countries that have:
    
    Above-average population
    Below-average area

In [8]:
%%sql
SELECT name, population, area AS area_sqkm, ROUND(CAST(population AS FLOAT) / area, 2) AS pop_area_ratio
FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts 
                    WHERE name <> 'World'
                   ) 
AND area < (SELECT AVG(area) 
            FROM facts
            WHERE name <> 'World'
           );

Done.


name,population,area_sqkm,pop_area_ratio
Bangladesh,168957745,148460,1138.07
Germany,80854408,357022,226.47
Iraq,37056169,438317,84.54
Italy,61855120,301340,205.27
Japan,126919659,377915,335.84
"Korea, South",49115196,99720,492.53
Morocco,33322699,446550,74.62
Philippines,100998376,300000,336.66
Poland,38562189,312685,123.33
Spain,48146134,505370,95.27


### FInding countries with the highest population/area ratio
The query will return countries in descending order by `pop_area_ratio` to showcase those countries with highest ratio.

In [9]:
%%sql
SELECT name, population, area AS area_sqkm, ROUND(CAST(population AS FLOAT) / area, 2) AS pop_area_ratio
FROM facts
WHERE name <> 'World'
ORDER BY pop_area_ratio DESC
LIMIT 10;

Done.


name,population,area_sqkm,pop_area_ratio
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


It is expected to find countries with small land masses and large populations to have the highest ratios.

### Finding the country with the largest population 

In [10]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population) FROM facts WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


### Finding the country with the highest growth rate

In [11]:
%%sql
SELECT *
FROM facts
WHERE population_growth = (SELECT MAX(population_growth) FROM facts WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


Reviewing the results from our summary statistics, we find that China and South Sudan are the countries with the largest population and highest growth rate, respectively.

### Exploring Ratios of Water and Land
From the cell directly above, area_land and area_water are `None` values for South Sudan implying that there may be more `None` values in our tables. Keep this in mind.

**Countries with highest water to land ratio**

In [12]:
%%sql
SELECT name, ROUND(CAST(area_water AS FLOAT) / area_land, 3) water_land_ratio
FROM facts
WHERE name <> 'World'
ORDER BY water_land_ratio DESC
LIMIT 10;

Done.


name,water_land_ratio
British Indian Ocean Territory,905.667
Virgin Islands,4.52
Puerto Rico,0.555
"Bahamas, The",0.387
Guinea-Bissau,0.285
Malawi,0.259
Netherlands,0.226
Uganda,0.223
Eritrea,0.164
Liberia,0.156


### Finding Which Countries Have More Water than Land

In [13]:
%%sql
SELECT name, area_land, area_water
FROM facts
WHERE name <> 'World'
AND area_water > area_land
ORDER BY area_water DESC;

Done.


name,area_land,area_water
British Indian Ocean Territory,60,54340
Virgin Islands,346,1564


### Which countries will add the most people to their populations next year?
Find the top 10 countries with population growth greater than the average population growth.

In [14]:
%%sql
SELECT name, population, population_growth
FROM facts
WHERE name <> 'World'
AND population_growth > (SELECT AVG(population_growth)
                         FROM facts
                         WHERE name <> 'World'
                        )
ORDER BY population_growth DESC
LIMIT 10;

Done.


name,population,population_growth
South Sudan,12042910,4.02
Malawi,17964697,3.32
Burundi,10742276,3.28
Niger,18045729,3.25
Uganda,37101745,3.24
Qatar,2194817,3.07
Burkina Faso,18931686,3.03
Mali,16955536,2.98
Cook Islands,9838,2.95
Iraq,37056169,2.93


### Which countries have a higher death rate than birth rate?

In [15]:
%%sql
SELECT name, death_rate, birth_rate
FROM facts
WHERE name <> 'World'
AND death_rate > birth_rate
ORDER BY death_rate DESC;

Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51


# Conclusion

In this introductory project to SQL, we have answered questions about the CIA Factbook and gained some insight into various aspects about the world.

Queries and subqueries were utilized to perform data aggregation to answer simulated ad-hoc requests.

Upon revisting, a closer look can be taken to determine what is classified as a country as certain territories like British Indian Ocean and Puerto Rico were included.